In [1]:
import numpy as np
import tensorflow as tf
import nnfs
from nnfs.datasets import spiral_data

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = x_train/255
X_test = x_test/255

flattened_images = [image.flatten() for image in X_train]
X_train = np.stack(flattened_images, axis=0)

flattened_images = [image.flatten() for image in X_test]
X_test = np.stack(flattened_images, axis=0)

In [4]:
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        self.weights = np.random.rand(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases
class Activation_ReLU:
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)
class Activation_SoftMax:
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities


In [14]:
# n_inputs = X_train[0].size
# X_train, y = spiral_data(100, 3)
hidden_neurons = 128
output_neurons = 10
input_neurons = 28*28
layer1 = Layer_Dense(input_neurons, hidden_neurons)
activation1 = Activation_ReLU()

layer2 = Layer_Dense(hidden_neurons, output_neurons)
activation2 = Activation_SoftMax()

layer1.forward(X_train)
activation1.forward(layer1.output)

layer2.forward(activation1.output)
activation2.forward(layer2.output)

print(activation2.output[0])
print(np.argmax(activation2.output[0]))
# y_train[0]

[5.78525950e-045 4.96734137e-106 1.23620402e-117 1.35166563e-181
 9.45259349e-097 2.61200560e-083 3.24041942e-147 2.35035382e-049
 9.99998356e-001 1.64392036e-006]
8


In [3]:
import numpy as np

# Define the sigmoid function for activation
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Define the derivative of the sigmoid function
def sigmoid_derivative(x):
    return x * (1 - x)

# Define the neural network class
class NeuralNetwork:
    def __init__(self, x, y):
        self.input = x
        self.weights1 = np.random.rand(self.input.shape[1], 4) # first layer with 4 neurons
        self.weights2 = np.random.rand(4, 1) # output layer with 1 neuron
        self.y = y
        self.output = np.zeros(self.y.shape)
    
    # Feed forward function
    def feedforward(self):
        self.layer1 = sigmoid(np.dot(self.input, self.weights1))
        self.output = sigmoid(np.dot(self.layer1, self.weights2))
    
    # Backpropagation function
    def backprop(self):
        d_weights2 = np.dot(self.layer1.T, (2 * (self.y - self.output) * sigmoid_derivative(self.output)))
        d_weights1 = np.dot(self.input.T, (np.dot(2 * (self.y - self.output) * sigmoid_derivative(self.output), self.weights2.T) * sigmoid_derivative(self.layer1)))
        
        # Update the weights with the calculated gradients
        self.weights1 += d_weights1
        self.weights2 += d_weights2
    
    # Train the model for a given number of iterations
    def train(self, iterations):
        for i in range(iterations):
            self.feedforward()
            self.backprop()
            print("iteration", i)
    
    # Make a prediction given new input data
    def predict(self, new_input):
        layer1 = sigmoid(np.dot(new_input, self.weights1))
        output = sigmoid(np.dot(layer1, self.weights2))
        return output


In [5]:
model = NeuralNetwork(X_train, y_train)
model.train(10)